### 01 packages

In [1]:
##########################################################################################
##########################################################################################

import matplotlib

import matplotlib.pyplot as plt

from matplotlib.ticker import MultipleLocator, FormatStrFormatter

##########################################################################################
##########################################################################################

import numpy as np

import pandas as pd

import random

##########################################################################################
##########################################################################################

from scipy.stats import truncnorm

from shapely.geometry import Point,Polygon

import networkx as nx

import math

##########################################################################################
##########################################################################################

import warnings

warnings.filterwarnings("ignore")

/Users/yyan2437/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### 02 parameters

In [6]:
##########################################################################################
##########################################################################################

instance='03'

lambda_value = 10

pattern='centralized'#'centralized','uniform'

##########################################################################################
##########################################################################################

### 03 data

In [7]:


##########################################################################################
##########################################################################################

Hexagons=np.load('./01network/'+instance+'/Hexagons.npy')

##########################################################################################
##########################################################################################

Hexagon_nodes=np.load("./01network/"+instance+"/Hexagon_nodes.npy",allow_pickle=True).item()

Hexagon_Center=np.load("./01network/"+"01"+"/Hexagon_Center.npy",allow_pickle=True).item()

Hexagons_Neighbors=np.load("./01network/"+instance+"/Hexagons_Neighbors.npy",allow_pickle=True).item()

##########################################################################################
##########################################################################################

Hexagon_Vertices=np.load("./01network/"+instance+"/Hexagon_Vertices.npy",allow_pickle=True).item()

##########################################################################################
##########################################################################################

Nodes=np.load("./01network/"+instance+"/Nodes.npy")

Edges=np.load("./01network/"+instance+"/Edges.npy")

##########################################################################################
##########################################################################################

Lockers=np.load("./01network/"+instance+"/Lockers.npy")

Lockers=tuple([(x[0],x[1]) for x in Lockers])

Locker_Hexagon=np.load("./01network/"+instance+"/Locker_Hexagon.npy",allow_pickle=True).item()

Hexagon_Locker=np.load("./01network/"+instance+"/Hexagon_Locker.npy",allow_pickle=True).item()

##########################################################################################
##########################################################################################

Road_Graph = nx.Graph()

for node in Nodes:
    
    node=tuple(node)
        
    Road_Graph.add_node(node)

for edge in Edges:
    
    node1=tuple(edge[0])
    
    node2=tuple(edge[1])
    
    Road_Graph.add_edge(node1, node2, weight=Point(node1).distance(Point(node2)))


##########################################################################################
##########################################################################################

Region_G = nx.Graph()

Region_G.add_nodes_from(Hexagons)

for hexagon_id in Hexagons_Neighbors:
    
    for neighor_id in Hexagons_Neighbors[hexagon_id]:
        
        Region_G.add_edge(hexagon_id, neighor_id)


##########################################################################################
##########################################################################################

parcel_df=pd.read_csv('./02data/01/instance_'+str(lambda_value)+'_'+pattern+'.csv')

parcel_df=parcel_df.drop(columns=['Unnamed: 0'])

parcel_df['pick_up']=parcel_df.apply(lambda x:tuple([round(float(y),2) for y in x.pick_up.replace('(','').replace(')','').split(',')]),axis=1)

parcel_df['drop_off']=parcel_df.apply(lambda x:tuple([round(float(y),2) for y in x.drop_off.replace('(','').replace(')','').split(',')]),axis=1)

def check_hexagon(node):
    
    for hexagon_id,nodes in Hexagon_nodes.items():
        
        if node in nodes:
            
            break
            
    return hexagon_id

parcel_df['h1']=parcel_df.apply(lambda x:check_hexagon(x.pick_up),axis=1)

parcel_df['h2']=parcel_df.apply(lambda x:check_hexagon(x.drop_off),axis=1)

##########################################################################################
##########################################################################################

parcel_df['shortest_region_path']=parcel_df.apply(lambda x:nx.shortest_path(Region_G, source=x.h1, target=x.h2),axis=1)

##########################################################################################
##########################################################################################

exploded_parcel_df = parcel_df.explode('shortest_region_path')

##########################################################################################
##########################################################################################

exploded_statistics=exploded_parcel_df.groupby('shortest_region_path').count()

exploded_statistics=exploded_statistics.reset_index()

exploded_statistics=exploded_statistics[['shortest_region_path','parcel_id']]

exploded_statistics=exploded_statistics.rename(columns={'shortest_region_path':'hexagon','parcel_id':'parcel_num'})

##########################################################################################
##########################################################################################

parcel_nums={h:0 for h in Hexagons}

for idx,row in exploded_statistics.iterrows():
    
    parcel_nums[row.hexagon]=row.parcel_num
    
allocations={h:1 for h in Hexagons}

while sum(allocations.values())<28:
    
    parcel_supply_ratio={h:parcel_nums[h]/allocations[h] for h in Hexagons}
    
    select_h=max(parcel_supply_ratio, key=lambda k: parcel_supply_ratio[k])
    
    allocations[select_h]+=1
    
##########################################################################################
##########################################################################################

 
courier_ids=list()

courier_nodes=list()

courier_hexagons=list()

idx=0

for hexagon in Hexagons:
    
    nums=allocations[hexagon]
    
    for i in range(nums):
        
        idx+=1
        
        courier_id='c'+str(idx)
        
        courier_ids.append(courier_id)
        
        courier_nodes.append(random.choice(Hexagon_nodes[hexagon]))
        
        courier_hexagons.append(hexagon)

courier_df=pd.DataFrame([])

courier_df['courier_id']=courier_ids

courier_df['courier_coord']=courier_nodes

courier_df['t']=0

courier_df['x']=courier_df.apply(lambda x:x.courier_coord[0],axis=1)

courier_df['y']=courier_df.apply(lambda x:x.courier_coord[1],axis=1)

courier_df=courier_df[['courier_id', 't', 'x', 'y']]

courier_df['hexagon']=courier_hexagons

##########################################################################################
##########################################################################################
    
courier_df.to_csv('./02data/'+instance+'/courier_instance'+str(lambda_value)+'_'+pattern+'.csv')

courier_df



,courier_id,t,x,y,hexagon
0,c1,0,1000.0,0.00,H1
1,c2,0,1000.0,0.00,H1
2,c3,0,1000.0,0.00,H1
3,c4,0,500.0,866.03,H1
4,c5,0,500.0,866.03,H1
5,c6,0,0.0,0.00,H1
6,c7,0,-500.0,-866.03,H1
7,c8,0,1000.0,0.00,H1
8,c9,0,0.0,0.00,H1
9,c10,0,500.0,-866.03,H1
